1.查询2024年star数量前10000的项目和star数量

In [ ]:
%%sql
SELECT
  repo.id AS repo_id,
  repo.name AS repo_name,
  COUNT(1) AS star_count
FROM
  `githubarchive.year.2024`
WHERE
  type = 'WatchEvent'
GROUP BY
  repo.id, repo.name
ORDER BY
  star_count DESC
LIMIT 10000;

2. 查询仓库的贡献者

In [ ]:
%%sql
SELECT
  original.type AS Type,
  repos.repo_id AS Repo_ID,
  repos.repo_name AS Repo_Name,
  original.actor.id AS contributor_ID,
  original.actor.login AS contributor,
FROM
  `github-network-465507.github.repos_top_10000_in_2024` AS repos
JOIN
  `githubarchive.year.2024` AS original
ON
  repos.repo_id = original.repo.id
WHERE
  original.type IN (
    'PushEvent',
    'PullRequestEvent',
    'PullRequestReviewEvent',
    'PullRequestReviewCommentEvent',
    'CommitCommentEvent',
    'GollumEvent',
    'IssuesEvent',
    'IssueCommentEvent',
    'CreateEvent',
    'ForkEvent',
    'ReleaseEvent'
  );

3. 预处理_删除部分BOTs

In [ ]:
%%sql
SELECT *
FROM `github-network-465507.github.Event_List` AS g
WHERE NOT REGEXP_CONTAINS(
  contributor,
  r'(\[bot\]$|(-?bots?$)|-robot$|QAbot|^robot-)'
);


4.预处理_获得每一个用户的贡献次数

In [ ]:
%%sql
SELECT
  Repo_ID,
  contributor_ID,
  contributor,
  COUNT(*) AS contribution_count
FROM `github-network-465507.github.EventList_withoutBots_Final`
GROUP BY Repo_ID, contributor_ID, contributor
ORDER BY contribution_count DESC;

5. 查找更多bot

In [21]:
import pandas as pd
import re
# 读入 CSV
df = pd.read_csv("Archived/Contributor_withContributonCount.csv")

patterns_1 = [
    r"bot",
    r"machine",
    r"automation",
    r"queue",
    r"testing",
    r"promoter",
    r"-ci$",
    r"\bci\b"
]

patterns = [

      r"bot$",

]

def is_bot(name):
    name_lower = str(name).lower()
    return any(re.search(p, name_lower) for p in patterns)

# 新增一列标记是否为机器人
df["is_bot"] = df["contributor"].apply(is_bot)

# 提取机器人名单
bots = df[df["is_bot"]]

# 按 contributor 分组，累加 contribution_count
bots_grouped = bots.groupby("contributor")["contribution_count"].sum().reset_index()

# 按贡献次数从高到低排序
bots_grouped_sorted = bots_grouped.sort_values(by="contribution_count", ascending=False)

# 打印结果
print("机器人名单及贡献次数（按贡献从高到低）：")
for _, row in bots_grouped_sorted.iterrows():
    print(f"{row['contributor']}")


机器人名单及贡献次数（按贡献从高到低）：
BrewTestBot
VSCodeTriageBot
OhMyGuus-Bot
PX4BuildBot
TooljetBot
DrahtBot
CKEditorBot
PckgrBot
PJBot
GiteaBot
MagiskBot
MelvinBot
JustArchi-ArchiBot
EgorBot
SDLWikiBot
prestonBot
iQQBot
OD-BOT
AppVeyorBot
SwiftLintBot
CarbonInfraBot
YamonBot
KernelPRBot
DronecodeBot
FishtownBuildBot
BuggieBot
daytonaBot
OptimismBot
XprobeBot
DilumAluthgeBot
FlysonBot
CultofVecna-Bot
SpacemacsBot
PylanceBot
MihuBot
HackathonBot
RocketmqRustBot
TanmoyTheBoT
NSCAP-BOT
StridBot
SonarrBot
HigherOrderBot
Z3R0-The-Bot
ALIENvsROBOT
SwifterSwiftBot
KeatonTheBot
XposedBot
RylanBot
Rust-Winget-Bot
Sparkle-Bot
pwshBot
WerWolvTranslationBot
UN-9BOT
DialogueBot
SACHIBOT
RunTheBot
wyjBot
SimRunBot
MitchellBot
UptimeKumaBot
CertainBot
Forty-Bot
MuGemBot
KabyleBOT
Ol1BoT
BoRoMiR-BOT
s750Bot
JusticeTechBot
MillsyBot
NewstellerBot
HaystackBot
RrZNBot
RockHuangBot
shadowsBot
AnshMusicBot
TurekBot
SprinkleBot
PebsonnyBot
Man-Bot
CommandoBot
elizaAiBot
DreadfulBot
MonicaBot
ASentientBot
AICodingBot
RoBob

6. 删除bot

In [ ]:
import pandas as pd

df = pd.read_csv("contributors_withContributionCount.csv)

# bots.txt，每行一个名字
with open("../data/bots.txt", "r", encoding="utf-8") as f:
    bot_list = [line.strip() for line in f if line.strip()]
df_cleaned = df[~df["contributor"].isin(bot_list)]

df_cleaned.to_csv("cleaned_file.csv", index=False)

print(f"清洗完成！原始数据 {len(df)} 行，删除机器人 {len(df) - len(df_cleaned)} 行，剩下 {len(df_cleaned)} 行。")

In [2]:
import pandas as pd
df_final = pd.read_parquet("repo_to_contribs.parquet")
print(df_final.head())

   Repo_ID                                       Contributors
0     3282  [48971777, 3788964, 80127367, 62702485, 294101...
1     3314  [40342677, 10594965, 81744029, 59172898, 25418...
2     8514  [12301, 8207, 51218, 4034581, 104796188, 64041...
3    34096  [65215500, 101272076, 3210258, 135803415, 1011...
4    36502  [12952068, 143310346, 27162132, 2201626, 89462...
